In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
df = pd.read_csv('amazon_products_sales_data_cleaned.csv')

In [3]:
df_model = df.dropna(subset=['number_of_reviews', 'current/discounted_price', 'rating', 'is_sponsored', 'is_best_seller']).copy()

# Convert Booleans to 0/1 (Integers)
df_model['is_sponsored'] = df_model['is_sponsored'].astype(int)
df_model['is_best_seller'] = df_model['is_best_seller'].astype(int)

# We log price and reviews to make them "normal" for the regression
df_model['log_price'] = np.log1p(df_model['current/discounted_price'])
df_model['log_reviews'] = np.log1p(df_model['number_of_reviews'])

In [4]:
# Select predictors
X = df_model[['log_price', 'rating', 'is_sponsored', 'is_best_seller']]
X = sm.add_constant(X) # Adds the intercept (beta_0)

# Calculate VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

print("--- Multicollinearity Check (VIF) ---")
print(vif_data)
# Rule: If VIF > 5, variables are too correlated. Ours should be fine (~1-2).

--- Multicollinearity Check (VIF) ---
          feature         VIF
0           const  182.585105
1       log_price    1.093912
2          rating    1.091387
3    is_sponsored    1.007451
4  is_best_seller    1.008491


In [5]:
# Define Target (Y) and Predictors (X)
Y = df_model['log_reviews']

# Fit the Model
model = sm.OLS(Y, X).fit()

# Print the "Money Shot" - The Statistical Summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            log_reviews   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.239
Method:                 Least Squares   F-statistic:                     2386.
Date:                Thu, 08 Jan 2026   Prob (F-statistic):               0.00
Time:                        18:09:09   Log-Likelihood:                -63581.
No. Observations:               30337   AIC:                         1.272e+05
Df Residuals:                   30332   BIC:                         1.272e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              7.3284      0.153     48.

In [11]:
# Get predictions and residuals
predictions = model.predict(X)
residuals = Y - predictions

# 1. Residual Plot (Check for Homoscedasticity)
plt.figure(figsize=(10, 5))
sns.scatterplot(x=predictions, y=residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted Log Reviews')
plt.ylabel('Residuals')
plt.title('Residuals vs. Fitted Values')
plt.savefig('Residuals vs. Fitted Values.png')
plt.close()

# 2. Q-Q Plot (Check for Normality of Errors)
fig = sm.qqplot(residuals, line='45', fit=True)
plt.title('Q-Q Plot of Residuals')
plt.savefig('Q-Q Plot of Residuals.png')
plt.close()